In [ ]:
import tensorflow as tf
import numpy as np

# ======================================================================
#  1. SETUP: Use a standard TF2 environment
# ======================================================================

# NO `tf.compat.v1.disable_eager_execution()` is needed.
# We are in the default TF2 eager mode.

SAVED_ENCODER_PATH = 'models/music_vae_encoder_keras'
TFLITE_MODEL_PATH = 'models/music_vae_encoder_tf2.tflite'

print(f"TensorFlow version: {tf.__version__}")
print(f"Eager execution enabled: {tf.executing_eagerly()}")


# ======================================================================
#  2. LOAD: Load the Keras model using the modern API
# ======================================================================

print(f"\nLoading Keras model from: {SAVED_ENCODER_PATH}")
# tf.keras in TF2 can load models saved from the TF1-era Keras.
try:
    standalone_encoder = tf.keras.models.load_model(SAVED_ENCODER_PATH)
    print("Model loaded successfully.")
    standalone_encoder.summary()
except Exception as e:
    print(f"Error loading model: {e}")
    print("This might happen if the model contains custom layers or legacy features not supported in TF2's loader.")
    # If this fails, you must fall back to the TF1 compatibility method.


# ======================================================================
#  3. CONVERT: Use the standard `TFLiteConverter`
# ======================================================================

print("\nStarting TFLite conversion using the TF2 converter...")

# The `from_keras_model` method is the most direct and recommended approach.
converter = tf.lite.TFLiteConverter.from_keras_model(standalone_encoder)

# --- APPLY THE FIX HERE ---
# 1. Enable the "Select TF Ops" feature.
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TFLite native ops.
    tf.lite.OpsSet.SELECT_TF_OPS     # Enable TF ops.
]

# 2. Disable the experimental flag that was causing the error.
converter._experimental_lower_tensor_list_ops = False
# --- END OF FIX ---

# (Optional but Recommended) Add optimizations.
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Perform the conversion.
tflite_model = converter.convert()

print("Conversion successful!")


# ======================================================================
#  4. SAVE AND VERIFY
# ======================================================================

with open(TFLITE_MODEL_PATH, 'wb') as f:
    f.write(tflite_model)

print(f"\nSaved TFLite model to: {TFLITE_MODEL_PATH}")

# Verification step remains the same and is always a good idea.
print("\nVerifying the TFLite model...")
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
tflite_output = interpreter.get_tensor(output_details[0]['index'])
print("TFLite model ran successfully!")
print("Sample output:", tflite_output[0, :5])

